In [66]:
%run -m ipy_startup
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from mgds.data_aggregation import config

In [63]:
def get_notebooks(regex):
    notebook_dir = config.IMPORT_NOTEBOOK_DIR
    
    # Find all ipynb files under notebook directory
    sources = !find $notebook_dir | grep "ipynb$" | grep -v ipynb_checkpoints
    notebooks = pd.Series(list(sources))
    
    # Filter to only notebook paths matching the given regex
    return list(notebooks[notebooks.str.contains(regex)])

In [69]:
#get_notebooks('gdsc/v1')
nb_files = get_notebooks('hugo')
nb_files

['/Users/eczech/repos/mgds/python/notebook/data_aggregation/import/hugo/v1/gene_meta.ipynb']

In [72]:
os.path.basename('/Users/eczech/repos/mgds/python/notebook/data_aggregation/import/hugo/v1/gene_meta.ipynb')

'gene_meta.ipynb'

In [75]:
def execute_notebook(nb_file, exec_processor):
    print('Executing notebook "{}"'.format(nb_file))
    
    exec_path = os.path.join(os.path.dirname(nb_file), 'exec')
    if not os.path.exists(exec_path):
        os.mkdir(exec_path)

    nb_file_out = os.path.join(exec_path, os.path.basename(nb_file))
    
    with open(nb_file) as f:
        nb = nbformat.read(f, as_version=4)

    try:
        out = ep.preprocess(nb, {'metadata': {'path': exec_path}})
    except CellExecutionError:
        msg = 'Error executing the notebook "%s".\n\n' % notebook_filename
        msg += 'See notebook "%s" for the traceback.' % notebook_filename_out
        print(msg)
        raise
    finally:
        print('Writing results to notebook "{}"'.format(nb_file_out))
        with open(nb_file_out, mode='wt') as f:
            nbformat.write(nb, f)
            
def execute_notebooks(nb_files):
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
    for nb_file in nb_files:
        execute_notebook(nb_file, ep)
        
execute_notebooks(nb_files)

Executing notebook "/Users/eczech/repos/mgds/python/notebook/data_aggregation/import/hugo/v1/gene_meta.ipynb"
Writing results to notebook "/Users/eczech/repos/mgds/python/notebook/data_aggregation/import/hugo/v1/exec/gene_meta.ipynb"
